In [1]:
import pickle
from natsort import natsorted

basedir = '../../ImageNomer/data/anton/cohorts/test'
demoname = f'{basedir}/demographics.pkl'

with open(demoname, 'rb') as f:
    demo = pickle.load(f)
    
subs = natsorted(list(demo['race'].keys()))
print(len(subs))

830


In [21]:
import numpy as np

task = 'rest'
pre = 'fc'

# Load FC and race

fcs = []
pncrace = []

for sub in subs:
    fname = f'{basedir}/{pre}/{sub}_task-{task}_{pre}.npy'
    if demo['race'][sub] == 'EA':
        pncrace.append(0)
    elif demo['race'][sub] == 'AA':
        pncrace.append(1)
    else:
        continue
    fc = np.load(fname)
    fcs.append(fc)
#     race.append(int(demo['sex'][sub] == 'M'))
    
fcs = np.stack(fcs)
pncrace = np.array(pncrace)
print(fcs.shape)
print(pncrace.shape)
print(np.mean(pncrace))
print(pncrace[0:3])

(733, 34716)
(733,)
0.44474761255115963
[0 0 0]


In [48]:
# Load BSNIP data

import pickle
import pandas as pd

# NaN FC: Baltimore_S3130FCP1 Detroit_1570_TR_S1570PVL1
badsub = ['Baltimore_S3130FCP1', 'Detroit_1570_TR_S1570PVL1']
bsnip = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

ts = pickle.load(open(f'{bsnip}/bsnip_rc_ts_centered.pkl', 'rb'))
# race = pickle.load(open('../../BSNIP/race.pkl', 'rb'))

for s in badsub:
    del ts[s]
#     del race[s]

# samsung = '/run/media/anton/Samsung_T5'
# bsnip = f'{samsung}/BSNIP'
pheno = f'{bsnip}/Phenotype'

iid = pd.read_csv(f'{pheno}/BSNIP_Imaging_ID.csv')
idb = pd.read_csv(f'{pheno}/BSNIP_Imaging_Database.csv')

print('Complete')

Complete


In [49]:
from math import isnan

def get_field(iid, idb, subid, field):
    subid = subid.split('_')[-1]
    pid = iid[iid['StudyID'] == subid]['pid']
    uid = iid[iid['StudyID'] == subid]['UID']
    ser2 = idb[idb['Scan_ID'] == subid]
    again = False
    try:
        if isnan(pid):
            again = True
        elif idb[idb['pid'] == int(pid)].shape[0] != 1:
            again = True
        else:
            ser = idb[idb['pid'] == int(pid)]
            return ser[field].item()
    except:
        again = True
    if again and ser2.shape[0] == 1:
        return ser2[field].item()
    return None

race = dict()

for k in ts:
    r = get_field(iid, idb, k, 'Race')
#     race[k] = r
    if r == 'AA' or r == 'CA':
        race[k] = r
        
print(len(list(race.keys())))

1165


In [52]:
import numpy as np

tx = []
ty = []

for k,v in race.items():
    if v == 'CA':
        ty.append(0)
    elif v == 'AA':
        ty.append(1)
    else:
        continue
    p = np.corrcoef(ts[k])
    a,b = np.triu_indices(264,1)
    tx.append(p[a,b])
        
tx = np.stack(tx)
ty = np.array(ty)

print(tx.shape)
print(ty.shape)

(1165, 34716)
(1165,)


In [19]:
print(1-np.mean(ty))

0.6678111587982833


In [61]:
# Linear

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

ntrain = 1000
rs = []
ts = []

for i in range(20):
    xtr, xt, ytr, yt = train_test_split(tx, ty, stratify=ty, train_size=ntrain)
    tr1 = np.where(ytr == 1)[0]
    tr0 = np.where(ytr == 0)[0][:len(tr1)]
    t1 = np.where(yt == 1)[0]
    t0 = np.where(yt == 0)[0][:len(t1)]
    xtr = np.concatenate([xtr[tr1], xtr[tr0]])
    xt = np.concatenate([xt[t1], xt[t0]])
    ytr = np.concatenate([ytr[tr1], ytr[tr0]])
    yt = np.concatenate([yt[t1], yt[t0]])
    print(xtr.shape)
    print(xt.shape)

    clf = LogisticRegression(C=1000, max_iter=1000).fit(xtr, ytr)
    yhat = clf.predict(xt)
    acc = np.mean((yhat == yt).astype("int"))
    acc = float(acc)
    print(acc)
    rs.append(acc)
#     yhat = clf.predict(tx)
#     acc = np.mean((yhat == ty).astype("int"))
#     acc = float(acc)
#     print(acc)
#     ts.append(acc)
    yhat = clf.predict(fcs)
    idcs = np.where(pncrace == 0)[0]
    acc = np.mean((yhat[idcs] == pncrace[idcs]).astype("int"))
    acc = float(acc)
    print(acc)
    ts.append(acc)
    
print('---')
print(np.mean(rs))
print(np.std(rs))
print(np.mean(ts))
print(np.std(ts))

(664, 34716)
(110, 34716)
0.8545454545454545
0.9041769041769042
(664, 34716)
(110, 34716)
0.7727272727272727
0.8968058968058968
(664, 34716)
(110, 34716)
0.7818181818181819
0.8918918918918919
(664, 34716)
(110, 34716)
0.8181818181818182
0.8353808353808354
(664, 34716)
(110, 34716)
0.7454545454545455
0.914004914004914
(664, 34716)
(110, 34716)
0.8363636363636363
0.9090909090909091
(664, 34716)
(110, 34716)
0.7454545454545455
0.8820638820638821
(664, 34716)
(110, 34716)
0.7363636363636363
0.9041769041769042
(664, 34716)
(110, 34716)
0.7636363636363637
0.9213759213759214
(664, 34716)
(110, 34716)
0.7454545454545455
0.9336609336609336
(664, 34716)
(110, 34716)
0.7818181818181819
0.9213759213759214
(664, 34716)
(110, 34716)
0.8272727272727273
0.8869778869778869
(664, 34716)
(110, 34716)
0.8545454545454545
0.8501228501228502
(664, 34716)
(110, 34716)
0.7636363636363637
0.9238329238329238
(664, 34716)
(110, 34716)
0.8181818181818182
0.8918918918918919
(664, 34716)
(110, 34716)
0.7818181818181

In [26]:
1-np.mean(pncrace)

0.5552523874488404

In [58]:
len(idcs)

326